## Calculate effect size

参考: https://mizumot.com/files/EffectSize_KELES31.pdf

重回帰モデルの偏相関係数に対する個別の検定(t検定) <br>
t検定に対する効果量$r$
$$ r = \sqrt{\frac{t^2}{t^2 + df}} $$

$t$: t値 <br>
$df$: 自由度(サンプル数 - 推定するパラメータ数) 

In [2]:
import pandas as pd
import openpyxl
import glob
import os

### Input Data

ファイル名の意味 <br>
\[atlas\]\_[model\]_\[covariates\].xlsx

例　BN_MR_Ag_Mr_Sb_Ti_Ge.xlsx <br>
- BN: BrainNetome
- MR: Multiple Regression
- Ag: age, Mr: MRI_performance, Sb: subCategory_performance, Ti: TIV, Ge: Gender

In [48]:
# optional
file_idx = 2

file_names = glob.glob('../corresponded/*.xlsx', recursive=True)
file_names

['../corresponded/cBN_MR_Ag_Mr_Sb_Ti_Ge.xlsx',
 '../corresponded/cBN_MR_Ag_Mr_Ti_Ge.xlsx',
 '../corresponded/cBN_MR_Ag_Sb_Ti_Ge.xlsx']

In [49]:
df = pd.read_excel(file_names[file_idx], header=1)
df

,0,P-value,T-value,Ze-value,BN_Atlas_246_2mm,0.1,0.2,0.3
0,1,0.002591,3.06339,2.79547,OI090,Temporal Lobe,"ITG, Inferior Temporal Gyrus","A20iv, intermediate ventral area 20"
1,2,0.006177,2.69662,2.50185,OI152,Parietal Lobe,"Pcun, Precuneus","dmPOS, dorsomedial parietooccipital sulcus(PEr)"
2,3,0.007923,2.58819,2.41244,OI094,Temporal Lobe,"ITG, Inferior Temporal Gyrus","A20r, rostral area 20"
3,4,0.023107,2.09759,1.99342,OI002,Frontal Lobe,"SFG, Superior Frontal Gyrus","A8m, medial area 8"
4,5,0.023824,2.08288,1.98050,OI110,Temporal Lobe,"PhG, Parahippocampal Gyrus","A35/36r, rostral area 35/36"
5,6,0.033308,1.91790,1.83426,OI079,Temporal Lobe,"STG, Superior Temporal Gyrus","A22r, rostral area 22"
6,7,0.034325,1.90277,1.82072,OI071,Temporal Lobe,"STG, Superior Temporal Gyrus","A41/42, area 41/42"
7,8,0.042834,1.78944,1.71871,OI012,Frontal Lobe,"SFG, Superior Frontal Gyrus","A9m,medial area 9"
8,9,0.046399,1.74765,1.68082,OI029,Frontal Lobe,"IFG, Inferior Frontal Gyrus","A44d,dorsal area 44"
9,10,0.049590,1.71251,1.64884,OI176,Limbic Lobe,"CG, Cingulate Gyrus","A23d, dorsal area 23"


### Calculate effect size

In [50]:
tValues = df.iloc[:,[2]]
tValues

#sample size: 30, number of parameters: 5(covarites + intercept)
effect_size = (tValues**2 / (tValues**2 + 25))**0.5
effect_size.columns = ['Effect size r']
df.insert(4, effect_size.columns[0], effect_size[effect_size.columns[0]])

df = df.rename(columns={df.columns[6]: 'Lobe', df.columns[7]: 'Gyrus', df.columns[8]: 'BA'})
df = df.drop(df.columns[0], axis=1)
df

,P-value,T-value,Ze-value,Effect size r,BN_Atlas_246_2mm,Lobe,Gyrus,BA
0,0.002591,3.06339,2.79547,0.522422,OI090,Temporal Lobe,"ITG, Inferior Temporal Gyrus","A20iv, intermediate ventral area 20"
1,0.006177,2.69662,2.50185,0.474688,OI152,Parietal Lobe,"Pcun, Precuneus","dmPOS, dorsomedial parietooccipital sulcus(PEr)"
2,0.007923,2.58819,2.41244,0.459701,OI094,Temporal Lobe,"ITG, Inferior Temporal Gyrus","A20r, rostral area 20"
3,0.023107,2.09759,1.99342,0.386855,OI002,Frontal Lobe,"SFG, Superior Frontal Gyrus","A8m, medial area 8"
4,0.023824,2.08288,1.98050,0.384544,OI110,Temporal Lobe,"PhG, Parahippocampal Gyrus","A35/36r, rostral area 35/36"
5,0.033308,1.91790,1.83426,0.358137,OI079,Temporal Lobe,"STG, Superior Temporal Gyrus","A22r, rostral area 22"
6,0.034325,1.90277,1.82072,0.355670,OI071,Temporal Lobe,"STG, Superior Temporal Gyrus","A41/42, area 41/42"
7,0.042834,1.78944,1.71871,0.336959,OI012,Frontal Lobe,"SFG, Superior Frontal Gyrus","A9m,medial area 9"
8,0.046399,1.74765,1.68082,0.329955,OI029,Frontal Lobe,"IFG, Inferior Frontal Gyrus","A44d,dorsal area 44"
9,0.049590,1.71251,1.64884,0.324024,OI176,Limbic Lobe,"CG, Cingulate Gyrus","A23d, dorsal area 23"


### Output Excel

In [51]:
save_dir = '../effect_size'
os.makedirs(save_dir, exist_ok=True)

tmp = pd.ExcelFile(file_names[file_idx])

df.to_excel(save_dir+'/e'+file_names[file_idx].split('/')[-1], sheet_name=tmp.sheet_names[0])